In [3]:
!pip install requests

In [5]:
import requests
import json
import pandas as pd

# Ripple API endpoint
url = "https://clio.altnet.rippletest.net:51234/"  # Alternative Ripple server

# First, get the latest validated ledger index
initial_payload = {
    "method": "ledger_current",
    "params": [{}]
}

# Send the request to get the latest validated ledger
response = requests.post(url, json=initial_payload)
latest_ledger_index = response.json().get("result", {}).get("ledger_current_index")

# Define the number of ledgers to retrieve
num_ledgers = 20

# Initialize list to store ledger data
ledger_data = []

# For loop to retrieve information for the last 3 ledgers
for i in range(num_ledgers):
    ledger_index = str(int(latest_ledger_index) - i)  # Move backwards in the ledger sequence
    print(f"Retrieving Ledger Index: {ledger_index}")

    # Create JSON request payload for the ledger
    payload = {
        "method": "ledger",
        "params": [
            {
                "ledger_index": ledger_index,
                "transactions": True,
                "expand": True
            }
        ]
    }

    # Send the request
    response = requests.post(url, json=payload)

    # Process the response
    try:
        response_json = response.json()  # Try to load the response as JSON
        ledger_info = response_json.get("result", {}).get("ledger", {})

        # Extract information for each transaction in the ledger
        if "transactions" in ledger_info:
            for tx in ledger_info["transactions"]:
                ledger_data.append({
                    "LedgerSequence": ledger_info.get("ledger_index"),
                    "CloseTime": ledger_info.get("close_time_human"),
                    "TransactionType": tx.get("TransactionType"),
                    "Account": tx.get("Account"),
                    "Fee": tx.get("Fee"),
                    "TransactionResult": tx.get("meta", {}).get("TransactionResult"),
                    "TakerGets": tx.get("TakerGets"),
                    "TakerPays": tx.get("TakerPays"),
                    "Issuer": tx.get("TakerPays", {}).get("issuer") if isinstance(tx.get("TakerPays"), dict) else None,
                    "Currency": tx.get("TakerPays", {}).get("currency") if isinstance(tx.get("TakerPays"), dict) else None,
                    "Sequence": tx.get("Sequence"),
                    "OfferSequence": tx.get("OfferSequence"),
                    "OwnerCount": tx.get("OwnerCount"),
                    "Flags": tx.get("Flags")
                })

    except json.JSONDecodeError:
        print(f"Error: The response for ledger {ledger_index} is not a valid JSON.")
        print("Raw response from server:")
        print(response.text)

# Convert the collected data to a pandas DataFrame
df = pd.DataFrame(ledger_data)

KeyboardInterrupt: 

In [24]:
import requests
import json
import pandas as pd
import concurrent.futures

# Ripple API endpoint
url = "https://testnet.xrpl-labs.com/"  # Alternative Ripple server

# First, get the latest validated ledger index
initial_payload = {
    "method": "ledger_current",
    "params": [{}]
}

# Send the request to get the latest validated ledger
response = requests.post(url, json=initial_payload)
latest_ledger_index = response.json().get("result", {}).get("ledger_current_index")

# Define the number of ledgers to retrieve
num_ledgers = 1000

# Initialize list to store ledger data
ledger_data = []

# Function to retrieve ledger information
def get_ledger_data(ledger_index):
    print(f"Retrieving Ledger Index: {ledger_index}")

    payload = {
        "method": "ledger",
        "params": [
            {
                "ledger_index": ledger_index,
                "transactions": True,
                "expand": True
            }
        ]
    }

    response = requests.post(url, json=payload)

    try:
        response_json = response.json()  # Try to load the response as JSON
        ledger_info = response_json.get("result", {}).get("ledger", {})
        data = []

        # Extract information for each transaction in the ledger
        if "transactions" in ledger_info:
            for tx in ledger_info["transactions"]:
                data.append({
                    "LedgerSequence": ledger_info.get("ledger_index"),
                    "CloseTime": ledger_info.get("close_time_human"),
                    "TransactionType": tx.get("TransactionType"),
                    "Account": tx.get("Account"),
                    "Fee": tx.get("Fee"),
                    "TransactionResult": tx.get("meta", {}).get("TransactionResult"),
                    "TakerGets": tx.get("TakerGets"),
                    "TakerPays": tx.get("TakerPays"),
                    "Issuer": tx.get("TakerPays", {}).get("issuer") if isinstance(tx.get("TakerPays"), dict) else None,
                    "Currency": tx.get("TakerPays", {}).get("currency") if isinstance(tx.get("TakerPays"), dict) else None,
                    "Sequence": tx.get("Sequence"),
                    "TicketSequence": tx.get("TicketSequence"),
                    "Signers": tx.get("Signers"),
                    "OfferSequence": tx.get("OfferSequence"),
                    "OwnerCount": tx.get("OwnerCount"),
                    "Flags": tx.get("Flags")
                })
        return data

    except json.JSONDecodeError:
        print(f"Error: The response for ledger {ledger_index} is not a valid JSON.")
        print("Raw response from server:")
        print(response.text)
        return []

# Use ThreadPoolExecutor to parallelize ledger retrieval
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(get_ledger_data, str(int(latest_ledger_index) - i)) for i in range(num_ledgers)]

    # Collect results as they complete
    for future in concurrent.futures.as_completed(futures):
        ledger_data.extend(future.result())  # Add the results to the ledger_data list

# Convert the collected data to a pandas DataFrame
df = pd.DataFrame(ledger_data)
print(df.head())


Retrieving Ledger Index: 758487Retrieving Ledger Index: 758486

Retrieving Ledger Index: 758485
Retrieving Ledger Index: 758484
Retrieving Ledger Index: 758483
Retrieving Ledger Index: 758482
Retrieving Ledger Index: 758481
Retrieving Ledger Index: 758480
Retrieving Ledger Index: 758479
Retrieving Ledger Index: 758478
Retrieving Ledger Index: 758477
Retrieving Ledger Index: 758476
Retrieving Ledger Index: 758475
Retrieving Ledger Index: 758474
Retrieving Ledger Index: 758473
Retrieving Ledger Index: 758472
Retrieving Ledger Index: 758471
Retrieving Ledger Index: 758470
Retrieving Ledger Index: 758469
Retrieving Ledger Index: 758468
Retrieving Ledger Index: 758467
Retrieving Ledger Index: 758466
Retrieving Ledger Index: 758465
Retrieving Ledger Index: 758464
Retrieving Ledger Index: 758463
Retrieving Ledger Index: 758462
Retrieving Ledger Index: 758461
Retrieving Ledger Index: 758460
Retrieving Ledger Index: 758459
Retrieving Ledger Index: 758458
Retrieving Ledger Index: 758457
Retrievi

In [23]:
df

,LedgerSequence,CloseTime,TransactionType,Account,Fee,TransactionResult,TakerGets,TakerPays,Issuer,Currency,Sequence,TicketSequence,Signers,OfferSequence,OwnerCount,Flags
0,758482,None,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,2233125,NaN,None,NaN,None,8.000000e+00
1,758482,None,Payment,rKhg1UjJrh1L26rSDpBgUdzJCZp8yv1zNN,15,None,None,None,None,None,758475,NaN,None,NaN,None,2.147484e+09
2,758482,None,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,2233126,NaN,None,NaN,None,8.000000e+00
3,758482,None,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,2233124,NaN,None,NaN,None,8.000000e+00
4,758482,None,Payment,rpYJB3SAYE3wR3ntQuMF1qUuC3gbkUg5uk,12,None,None,None,None,None,570993,NaN,None,NaN,None,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,758388,2024-Sep-16 09:29:10.000000000 UTC,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,2232757,NaN,None,NaN,None,8.000000e+00
465,758388,2024-Sep-16 09:29:10.000000000 UTC,Payment,rPT1Sjq2YGrBMTttX4GZHjKu9dyfzbpAYe,12,None,None,None,None,None,0,229360.0,None,NaN,None,0.000000e+00
466,758388,2024-Sep-16 09:29:10.000000000 UTC,Payment,rL4VhiMjkEKJP99Va8kpTLNYzqetWm58sp,12,None,None,None,None,None,6739,NaN,None,NaN,None,NaN
467,758388,2024-Sep-16 09:29:10.000000000 UTC,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,2232760,NaN,None,NaN,None,8.000000e+00


In [14]:
df.dtypes

LedgerSequence       object
CloseTime            object
TransactionType      object
Account              object
Fee                  object
TransactionResult    object
TakerGets            object
TakerPays            object
Issuer               object
Currency             object
Sequence              int64
OfferSequence        object
OwnerCount           object
Flags                 int64
dtype: object

In [18]:
# convert columns
df['LedgerSequence'] = pd.to_numeric(df['LedgerSequence'])
df['CloseTime'] = pd.to_datetime(df['CloseTime'])
df['Fee'] = pd.to_numeric(df['Fee'])
df['OfferSequence'] = pd.to_numeric(df['OfferSequence'])
df['OwnerCount'] = pd.to_numeric(df['OwnerCount'])


In [19]:
df.dtypes

LedgerSequence                     int64
CloseTime            datetime64[ns, UTC]
TransactionType                   object
Account                           object
Fee                                int64
TransactionResult                 object
TakerGets                         object
TakerPays                         object
Issuer                            object
Currency                          object
Sequence                           int64
OfferSequence                    float64
OwnerCount                       float64
Flags                              int64
dtype: object

In [21]:
df.describe()

,LedgerSequence,Fee,Sequence,OfferSequence,OwnerCount,Flags
count,43.000000,43.000000,4.300000e+01,0.0,0.0,43.000000
mean,523849.767442,10.139535,1.359158e+06,NaN,NaN,7.441860
std,2.966778,0.515539,3.766258e+05,NaN,NaN,2.062157
min,523845.000000,10.000000,0.000000e+00,NaN,NaN,0.000000
25%,523847.000000,10.000000,1.461082e+06,NaN,NaN,8.000000
50%,523850.000000,10.000000,1.461093e+06,NaN,NaN,8.000000
75%,523852.000000,10.000000,1.461104e+06,NaN,NaN,8.000000
max,523854.000000,12.000000,1.461114e+06,NaN,NaN,8.000000
